In [1]:
from pathlib import Path
import torch
from PIL import Image
import numpy as np
from test_quantize import SimpleTrainer2d 
import os
import random
import argparse

args = argparse.Namespace(
    data_name="kodak",
    dataset="/root/datasets/kodak",
    lr=1e-3,
    iterations=50000,
    model_name="GaussianImage_Cholesky",
    num_points=1000,
    model_path="./checkpoints_quant/kodak/GaussianImage_Cholesky_50000_1000",
    seed=1,
    quantize=True,
    save_imgs=True,
)

if args.seed is not None:
    torch.manual_seed(args.seed)
    random.seed(args.seed)
    torch.cuda.manual_seed(args.seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(args.seed)

data_path = os.path.expandvars("$HOME/data/kodak")
images = []
images.append(os.path.join(data_path, "kodim01.png"))
images.append(os.path.join(data_path, "kodim02.png"))

model_path1 = "/root/Dev/GaussianImage/checkpoints_quant/kodak/GaussianImage_Cholesky_50000_1000/kodim01/gaussian_model.best.pth.tar"
model_path2 = "/root/Dev/GaussianImage/checkpoints_quant/kodak/GaussianImage_Cholesky_50000_1000/kodim02/gaussian_model.best.pth.tar"
model = [model_path1, model_path2]


In [2]:
from torchvision.utils import save_image
data, comp = [], []

def test_image (image_path, model_path):
    trainer = SimpleTrainer2d(image_path=image_path, num_points=1000, 
        iterations=50000, model_path=model_path, args=args)

    data_dict, comp_dict = trainer.test()
    data.append(data_dict)
    comp.append(comp_dict)

def test_encoding(model_path, encoding):
    trainer = SimpleTrainer2d(num_points=1000, 
        iterations=50000, model_path=model_path, image_path = images[0], args=args)
    (result, img) = trainer.test_encoding_dict(encoding)
    save_image(img, "reconstructed.png")
    return result

In [3]:
for i in range(1):
    test_image(images[i], model[i])

for i in range(1):
    path = f"./result/comp_{i}.pth"
    torch.save(comp[i], path)
    for key, value in comp[i].items():
        if torch.is_tensor(value):
            print(f"{key}: {value}",end="\n")
        else:
            print(f"{key}: {type(value)}",end="\n")
    
    print("\n")

loading model path:/root/Dev/GaussianImage/checkpoints_quant/kodak/GaussianImage_Cholesky_50000_1000/kodim01/gaussian_model.best.pth.tar
Eval time:0.00023557s, FPS:4244.9462
PSNR:21.9997, MS_SSIM:0.766785, bpp:0.1468
position_bpp:0.0814, cholesky_bpp:0.0463, feature_dc_bpp:0.0192
xyz: tensor([[ 0.6919, -0.5879],
        [-0.1826,  0.5303],
        [-1.7207,  0.8188],
        ...,
        [-0.6523,  1.4004],
        [-0.1433,  0.3350],
        [-0.4336, -0.4902]], device='cuda:0', dtype=torch.float16)
feature_dc_index: tensor([[7, 1],
        [5, 7],
        [1, 1],
        ...,
        [2, 1],
        [7, 1],
        [2, 1]], device='cuda:0')
quant_cholesky_elements: tensor([[22., 12., 16.],
        [ 5., 19., 61.],
        [ 9., 28., 49.],
        ...,
        [ 6., 18.,  3.],
        [13., 17., 17.],
        [56., 16.,  6.]], device='cuda:0')




In [4]:
encoding = torch.load('./recon_output.pth')
encoding

{'xyz': tensor([[ 0.8021, -0.5255],
         [ 0.2202,  0.5613],
         [-1.5981,  0.8515],
         ...,
         [-0.7249,  1.1562],
         [ 0.0927,  0.2250],
         [-0.2752, -0.6822]], device='cuda:0'),
 'feature_dc_index': tensor([[7, 1],
         [5, 7],
         [1, 1],
         ...,
         [2, 1],
         [7, 1],
         [2, 1]], device='cuda:0', dtype=torch.int32),
 'quant_cholesky_elements': tensor([[22., 12., 16.],
         [ 5., 19., 61.],
         [ 9., 28., 49.],
         ...,
         [ 6., 18.,  3.],
         [13., 17., 17.],
         [56., 16.,  6.]], device='cuda:0')}

In [5]:
res = test_encoding(model[0], encoding)
res

loading model path:/root/Dev/GaussianImage/checkpoints_quant/kodak/GaussianImage_Cholesky_50000_1000/kodim01/gaussian_model.best.pth.tar
Eval time:0.00023520s, FPS:4251.7451
PSNR:10.9323, MS_SSIM:0.086508, bpp:0.1468
position_bpp:0.0814, cholesky_bpp:0.0463, feature_dc_bpp:0.0192


{'bpp': np.float64(0.14680989583333334),
 'position_bpp': 0.08138020833333333,
 'cholesky_bpp': 0.0462646484375,
 'feature_dc_bpp': np.float64(0.0191650390625),
 'psnr': 10.932267829696396,
 'ms-ssim': 0.08650793880224228,
 'rendering_time': 0.0002351975440979004,
 'rendering_fps': 4251.7450759764415}